<a href="https://colab.research.google.com/github/ghommidhWassim/GNN-variants/blob/main/graphSaint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!pip install torchvision
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
2.6.0+cu124
12.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    

In [2]:
# !pip install torch torchvision torchaudio --quiet
# !pip install scipy numpy --quiet
# !git clone https://github.com/graphsaint/graphsaint.git  # if you want to use official repo, or upload your own files

import os
import numpy as np
import scipy.sparse as sp
import torch
import time
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.datasets import Planetoid, Amazon
from torch_geometric.transforms import NormalizeFeatures, RandomNodeSplit
import torch_geometric.transforms as T
from torch_geometric.utils import to_scipy_sparse_matrix
from torch_geometric.loader import GraphSAINTNodeSampler, GraphSAINTEdgeSampler, GraphSAINTRandomWalkSampler
from sklearn.metrics import f1_score       # add this import once, near the top
import torch.nn as nn
import json
# (You will have to upload or place the GraphSAINT modules or install them if available)
# For simplicity, assume graphsaint package is already in your environment or uploaded as files.


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ------------------- Load Dataset -------------------

def clean_gpu_memory():
    """Cleans GPU memory without fully resetting the CUDA context"""
    import gc
    gc.collect()  # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # PyTorch cache
        torch.cuda.reset_peak_memory_stats()  # Reset tracking
        print(f"Memory after cleanup: {torch.cuda.memory_allocated()/1024**2:.2f} MB")


Compute GraphSAINT normalization: : 198778it [00:00, 927460.84it/s]                          
Compute GraphSAINT normalization: : 198592it [03:03, 1085.04it/s]                          
Compute GraphSAINT normalization: : 197445it [00:00, 2301321.51it/s]        


In [4]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = dropout

    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=-1)

NameError: name 'num_features' is not defined

In [5]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.edge_norm)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# ------------------- Evaluation Function -------------------
@torch.no_grad()
#def test(model, full_data):
 #   model.eval()
  #  out = model(full_data.x.to(device), full_data.edge_index.to(device))
  #  pred = out.argmax(dim=1)
  #  correct = (pred[full_data.test_mask] == full_data.y[full_data.test_mask].to(device)).sum()
  #  acc = int(correct) / int(full_data.test_mask.sum())
   # return acc

def test(model, full_data):
    model.eval()
    out = model(full_data.x.to(device), full_data.edge_index.to(device))

    # predictions for every node
    pred = out.argmax(dim=1)

    # isolate test nodes (keep everything on the SAME device, then move to CPU once)
    test_mask = full_data.test_mask
    y_true = full_data.y[test_mask].cpu()
    y_pred = pred[test_mask].cpu()

    # accuracy
    acc = (y_pred == y_true).float().mean().item()

    # micro‑F1
    f1_micro = f1_score(y_true.numpy(), y_pred.numpy(), average='micro')

    return acc, f1_micro
# ------------------- Run Training -------------------
def run(loader, method_name):
    # Use the global num_classes variable instead of trying to access it from the data object


    for epoch in range(1, 101):
        loss = train(model, loader, optimizer, criterion)
        print(f'{method_name} | Epoch {epoch:03d}, Loss: {loss:.4f}')

    acc,test_f1 = test(model, data)
    print(f'{method_name} | Final Test Accuracy: {acc:.4f} | Test F1 (micro): {test_f1:.4f}")')
    return acc,test_f1



In [ ]:
clean_gpu_memory()
def dataset_load():
    print(f"Using device: {device}")
    dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())
    data = dataset[0].to(device)
    return dataset.num_features, data, dataset.num_classes

num_features, data, num_classes = dataset_load()
data = data.cpu()
#loader_SAINT_256_node = GraphSAINTNodeSampler(data, batch_size=500, num_steps=4, sample_coverage=10)

#loader_SAINT_256_edge = GraphSAINTEdgeSampler(data, batch_size=500, num_steps=4, sample_coverage=10)

loader_SAINT_256_RW = GraphSAINTRandomWalkSampler(data, batch_size=128, walk_length=2, num_steps=4, sample_coverage=10)
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
model = GCN(
        in_channels=num_features,
        hidden_channels=64,
        out_channels=num_classes
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
# ------------------- Execute Training for Each Loader -------------------

#run(loader_SAINT_256_node, "GraphSAINT-NodeSampler")

#run(loader_SAINT_256_edge, "GraphSAINT-EdgeSampler")
start_time = time.time()

test_acc,f1_micro =  run(loader_SAINT_256_RW, "GraphSAINT-RandomWalkSampler")
end_time = time.time()


Memory after cleanup: 32.35 MB
Using device: cuda


Compute GraphSAINT normalization: : 197171it [00:00, 700366.12it/s]                          


Current GPU memory: 32.35 MB
Max GPU memory used: 71.91 MB
GraphSAINT-RandomWalkSampler | Epoch 001, Loss: 1.0769
GraphSAINT-RandomWalkSampler | Epoch 002, Loss: 1.0295
GraphSAINT-RandomWalkSampler | Epoch 003, Loss: 1.0015
GraphSAINT-RandomWalkSampler | Epoch 004, Loss: 0.9710
GraphSAINT-RandomWalkSampler | Epoch 005, Loss: 0.9126
GraphSAINT-RandomWalkSampler | Epoch 006, Loss: 0.8690
GraphSAINT-RandomWalkSampler | Epoch 007, Loss: 0.8127
GraphSAINT-RandomWalkSampler | Epoch 008, Loss: 0.8003
GraphSAINT-RandomWalkSampler | Epoch 009, Loss: 0.7396
GraphSAINT-RandomWalkSampler | Epoch 010, Loss: 0.7762
GraphSAINT-RandomWalkSampler | Epoch 011, Loss: 0.6972
GraphSAINT-RandomWalkSampler | Epoch 012, Loss: 0.6855
GraphSAINT-RandomWalkSampler | Epoch 013, Loss: 0.6769
GraphSAINT-RandomWalkSampler | Epoch 014, Loss: 0.6255
GraphSAINT-RandomWalkSampler | Epoch 015, Loss: 0.6379
GraphSAINT-RandomWalkSampler | Epoch 016, Loss: 0.6497
GraphSAINT-RandomWalkSampler | Epoch 017, Loss: 0.6833
GraphS

In [ ]:
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")

Current GPU memory: 17.72 MB
Max GPU memory used: 121.68 MB


In [ ]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "graphSAINT",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":71.91
}

with open("graphSAINT_rw_pubmed_results.json", "w") as f:
    json.dump(metrics, f)

**Cora**

In [ ]:
clean_gpu_memory()
def dataset_load():
    print(f"Using device: {device}")
    dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
    data = dataset[0].to(device)
    return dataset.num_features, data, dataset.num_classes

num_features, data, num_classes = dataset_load()
data = data.cpu()
#loader_SAINT_256_node = GraphSAINTNodeSampler(data, batch_size=128, num_steps=4, sample_coverage=10)

#loader_SAINT_256_edge = GraphSAINTEdgeSampler(data, batch_size=500, num_steps=4, sample_coverage=10)

loader_SAINT_256_RW = GraphSAINTRandomWalkSampler(data, batch_size=128, walk_length=2, num_steps=4, sample_coverage=10)
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
model = GCN(
        in_channels=num_features,
        hidden_channels=64,
        out_channels=num_classes
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
# ------------------- Execute Training for Each Loader -------------------

#run(loader_SAINT_256_node, "GraphSAINT-NodeSampler")
#run(loader_SAINT_256_edge, "GraphSAINT-EdgeSampler")

start_time = time.time()

test_acc,f1_micro =  run(loader_SAINT_256_RW, "GraphSAINT-RandomWalkSampler")
end_time = time.time()


Memory after cleanup: 34.56 MB
Using device: cuda


Compute GraphSAINT normalization: : 27920it [00:00, 754882.79it/s]         

Current GPU memory: 34.56 MB
Max GPU memory used: 49.56 MB
GraphSAINT-RandomWalkSampler | Epoch 001, Loss: 1.9206
GraphSAINT-RandomWalkSampler | Epoch 002, Loss: 1.8519
GraphSAINT-RandomWalkSampler | Epoch 003, Loss: 1.7789


GraphSAINT-RandomWalkSampler | Epoch 004, Loss: 1.7180
GraphSAINT-RandomWalkSampler | Epoch 005, Loss: 1.7123
GraphSAINT-RandomWalkSampler | Epoch 006, Loss: 1.6454
GraphSAINT-RandomWalkSampler | Epoch 007, Loss: 1.5716
GraphSAINT-RandomWalkSampler | Epoch 008, Loss: 1.5109
GraphSAINT-RandomWalkSampler | Epoch 009, Loss: 1.4647
GraphSAINT-RandomWalkSampler | Epoch 010, Loss: 1.3780
GraphSAINT-RandomWalkSampler | Epoch 011, Loss: 1.3111
GraphSAINT-RandomWalkSampler | Epoch 012, Loss: 1.2841
GraphSAINT-RandomWalkSampler | Epoch 013, Loss: 1.2303
GraphSAINT-RandomWalkSampler | Epoch 014, Loss: 1.1116
GraphSAINT-RandomWalkSampler | Epoch 015, Loss: 1.0837
GraphSAINT-RandomWalkSampler | Epoch 016, Loss: 1.0886
GraphSAINT-RandomWalkSampler | Epoch 017, Loss: 1.0257
GraphSAINT-RandomWalkSampler | Epoch 018, Loss: 0.9950
GraphSAINT-RandomWalkSampler | Epoch 019, Loss: 0.9533
GraphSAINT-RandomWalkSampler | Epoch 020, Loss: 0.9064
GraphSAINT-RandomWalkSampler | Epoch 021, Loss: 0.9116
GraphSAINT

In [ ]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "graphSAINT",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":49.56
}

with open("graphSAINT_rw_Cora_results.json", "w") as f:
    json.dump(metrics, f)

**Cetiser**

In [ ]:
clean_gpu_memory()
def dataset_load():
    print(f"Using device: {device}")
    dataset = Planetoid(root='data/Planetoid', name='CiteSeer', transform=NormalizeFeatures())
    data = dataset[0].to(device)
    return dataset.num_features, data, dataset.num_classes

num_features, data, num_classes = dataset_load()
data = data.cpu()
#loader_SAINT_256_node = GraphSAINTNodeSampler(data, batch_size=500, num_steps=4, sample_coverage=10)

#loader_SAINT_256_edge = GraphSAINTEdgeSampler(data, batch_size=128, num_steps=4, sample_coverage=10)

loader_SAINT_256_RW = GraphSAINTRandomWalkSampler(data, batch_size=128, walk_length=2, num_steps=4, sample_coverage=10)
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
model = GCN(
        in_channels=num_features,
        hidden_channels=64,
        out_channels=num_classes
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
# ------------------- Execute Training for Each Loader -------------------

#run(loader_SAINT_256_node, "GraphSAINT-NodeSampler")

# run(loader_SAINT_256_edge, "GraphSAINT-EdgeSampler")

start_time = time.time()

test_acc,f1_micro =  run(loader_SAINT_256_RW, "GraphSAINT-RandomWalkSampler")
end_time = time.time()


Compute GraphSAINT normalization:   0%|          | 0/137520 [14:05<?, ?it/s]


Memory after cleanup: 30.94 MB
Using device: cuda


Compute GraphSAINT normalization: : 34438it [00:00, 964641.18it/s]         


Current GPU memory: 30.94 MB
Max GPU memory used: 78.11 MB
GraphSAINT-RandomWalkSampler | Epoch 001, Loss: 1.7738
GraphSAINT-RandomWalkSampler | Epoch 002, Loss: 1.7283
GraphSAINT-RandomWalkSampler | Epoch 003, Loss: 1.6956
GraphSAINT-RandomWalkSampler | Epoch 004, Loss: 1.6590
GraphSAINT-RandomWalkSampler | Epoch 005, Loss: 1.6234
GraphSAINT-RandomWalkSampler | Epoch 006, Loss: 1.6000
GraphSAINT-RandomWalkSampler | Epoch 007, Loss: 1.5372
GraphSAINT-RandomWalkSampler | Epoch 008, Loss: 1.4884
GraphSAINT-RandomWalkSampler | Epoch 009, Loss: 1.4561
GraphSAINT-RandomWalkSampler | Epoch 010, Loss: 1.4074
GraphSAINT-RandomWalkSampler | Epoch 011, Loss: 1.3485
GraphSAINT-RandomWalkSampler | Epoch 012, Loss: 1.2784
GraphSAINT-RandomWalkSampler | Epoch 013, Loss: 1.2542
GraphSAINT-RandomWalkSampler | Epoch 014, Loss: 1.2034
GraphSAINT-RandomWalkSampler | Epoch 015, Loss: 1.2021
GraphSAINT-RandomWalkSampler | Epoch 016, Loss: 1.1122
GraphSAINT-RandomWalkSampler | Epoch 017, Loss: 1.1206
GraphS

In [ ]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "graphSAINT",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":78.11
}

with open("graphSAINT_rw_CiteSeer_results.json", "w") as f:
    json.dump(metrics, f)

**Amazon**

In [8]:
clean_gpu_memory()
def dataset_load():
  print(f"Using device: {device}")
  dataset = Amazon(
        root='data/Amazon',
        name='Computers',
        transform=T.Compose([
        NormalizeFeatures(),          # feature‑wise ℓ₂ normalisation
        RandomNodeSplit(              # ⇦ add a split transform
                split='train_rest',       # 10% val, 10% test by default
                num_val=0.1,
                num_test=0.1,
                num_splits=1,
            )
        ])
    )
  num_features = dataset.num_features
  num_classes = dataset.num_classes
  data = dataset[0].to(device)  # Get the first graph object.
  return num_features, data, num_classes, device,dataset

num_features, data, num_classes, device, dataset = dataset_load()
data = data.cpu()
#loader_SAINT_256_node = GraphSAINTNodeSampler(data, batch_size=500, num_steps=4, sample_coverage=10)

#loader_SAINT_256_edge = GraphSAINTEdgeSampler(data, batch_size=128, num_steps=4, sample_coverage=10)

loader_SAINT_256_RW = GraphSAINTRandomWalkSampler(data, batch_size=128, walk_length=2, num_steps=4, sample_coverage=10)
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
model = GCN(
        in_channels=num_features,
        hidden_channels=64,
        out_channels=num_classes
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
# ------------------- Execute Training for Each Loader -------------------

#run(loader_SAINT_256_node, "GraphSAINT-NodeSampler")

# run(loader_SAINT_256_edge, "GraphSAINT-EdgeSampler")
res=[]
training=[]
for i in range(10):
  start_time = time.time()

  test_acc,f1_micro = run(loader_SAINT_256_RW, "GraphSAINT-RandomWalkSampler")
  end_time = time.time()
  res.append(test_acc)
  training.append(end_time - start_time)

Memory after cleanup: 17.01 MB
Using device: cuda


Compute GraphSAINT normalization: : 138309it [00:00, 711678.76it/s]                         


Current GPU memory: 17.01 MB
Max GPU memory used: 64.90 MB
GraphSAINT-RandomWalkSampler | Epoch 001, Loss: 2.2628
GraphSAINT-RandomWalkSampler | Epoch 002, Loss: 2.0962
GraphSAINT-RandomWalkSampler | Epoch 003, Loss: 2.0598
GraphSAINT-RandomWalkSampler | Epoch 004, Loss: 1.9924
GraphSAINT-RandomWalkSampler | Epoch 005, Loss: 1.8832
GraphSAINT-RandomWalkSampler | Epoch 006, Loss: 1.9479
GraphSAINT-RandomWalkSampler | Epoch 007, Loss: 1.9000
GraphSAINT-RandomWalkSampler | Epoch 008, Loss: 1.9084
GraphSAINT-RandomWalkSampler | Epoch 009, Loss: 1.9198
GraphSAINT-RandomWalkSampler | Epoch 010, Loss: 1.9218
GraphSAINT-RandomWalkSampler | Epoch 011, Loss: 1.9010
GraphSAINT-RandomWalkSampler | Epoch 012, Loss: 1.8456
GraphSAINT-RandomWalkSampler | Epoch 013, Loss: 1.8747
GraphSAINT-RandomWalkSampler | Epoch 014, Loss: 1.9224
GraphSAINT-RandomWalkSampler | Epoch 015, Loss: 2.0195
GraphSAINT-RandomWalkSampler | Epoch 016, Loss: 1.8235
GraphSAINT-RandomWalkSampler | Epoch 017, Loss: 1.9108
GraphS

In [9]:
std_acc = np.std(res)
mean_test_acc = np.mean(res)
print(f"Test Accuracy: {mean_test_acc:.4f} ± {std_acc:.4f}")
std_training = np.std(training)
mean_training = np.mean(training)
print(f"training time: {mean_training:.4f} ± {std_training:.4f}")

Test Accuracy: 0.5201 ± 0.0786
training time: 2.1675 ± 0.2542


In [ ]:
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "graphSAINT",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "mem_MB":79.58
}

with open("graphSAINT_rw_amazon_results.json", "w") as f:
    json.dump(metrics, f)